In [1]:
import pandas as pd
import numpy as np
import random
import pickle

In [2]:
data_orig= pd.read_csv('../MA_PREDICTOR/data/ma_detailed_data_car.csv', parse_dates=['announcement_date'])

In [3]:
# Removing outliers
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [4]:
data.head()

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,shares_acquired,consideration_offered,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,100.00,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854
1,1629274020,2005-01-04,153.67,6166.05,SuperValu Inc,SVU,Total Logistics Inc,United States,United States,Public,...,100.00,Cash Only|Cash,Friendly,Tender Offer|Tender Merger,Lazard,William Blair & Co,Strengthen existing operations/expand presence...,54301020,52405030,-0.009362
2,1631278020,2005-01-11,89.16,11127.95,Fulton Financial Corp,FULT,"SVB Financial Services Inc, Sommerville,New Je...",United States,United States,Public,...,100.00,Cash|Cash|Common Stock|Common Stock,Friendly,Stock Swap,NaN,Danielson Associates Inc,Expand presence in new/foreign markets,55101010,55101010,-0.011249
3,1631266040,2005-01-12,427.54,1116.90,Cleveland-Cliffs Inc,CLF,Portman Ltd,Australia,United States,Public,...,80.39,Cash|Cash,Friendly,Tender Offer,Wilson HTM|Fleet National Bank|Fifth Third Bank,Gresham Partners|Azure Capital,Strengthen existing operations/expand presence...,51201020,51201020,0.013061
4,1632176020,2005-01-14,16.00,1722.27,Thomas & Betts Corp,TNB,Southern Monopole & Utilities Co,United States,United States,Subsidiary,...,100.00,Cash|Cash Only,Friendly,Divestiture,NaN,NaN,Acquire competitors technology/strategic assets,52102030,51201020,0.000000


In [5]:
data.columns

Index(['id', 'announcement_date', 'deal_value', 'acquiror_total_assets',
       'acquiror_name', 'acquiror_ticker', 'target_name', 'target_nation',
       'acquiror_nation', 'target_status', 'acquiror_industry',
       'target_industry', 'shares_at_announcement', 'shares_acquired',
       'consideration_offered', 'attitude', 'acquisition_technique',
       'acquiror_financial_advisor', 'target_financial_advisor', 'purpose',
       'acquiror_code', 'target_code', 'car'],
      dtype='object')

In [6]:
# Creating new dataframe to work on

clean = data[['announcement_date',
              'deal_value', 'acquiror_total_assets', # financials
              'target_nation', 'acquiror_nation', 'target_status', # participants
              'acquiror_code', 'target_code', # participants
              'shares_at_announcement', 'shares_acquired', # deal
              'consideration_offered', 'acquisition_technique', 'purpose', # deal 
              'acquiror_financial_advisor', 'target_financial_advisor', # deal
              'car'
             ]].copy()

# announcement_date (month)

In [7]:
clean['month']= pd.DatetimeIndex(clean['announcement_date']).month

# deal_value (relative deal_value)

In [8]:
clean['rel_deal_value'] = clean.apply(lambda row:
                                       row.deal_value / row.acquiror_total_assets
                                       if row.acquiror_total_assets # one observation has 0 total assets and
                                       else -1000, # will be filtered out
                                       axis=1)

# Filtering out acquiror_total_assets = 0
clean = clean[clean.rel_deal_value > 0]

# Dropping cols
clean.drop(columns=['deal_value', 'acquiror_total_assets'], inplace=True)

# acquiror_nation & target_nation (cross-border)

In [9]:
clean['cross_border'] = clean.apply(lambda row:
                                    'cross_border'
                                    if row.acquiror_nation == row.target_nation
                                    else 'national',
                                    axis=1)
# Dropping cols
clean.drop(columns=['acquiror_nation', 'target_nation'], inplace=True)

# target_status

In [10]:
clean['target_status'] = clean['target_status'].apply(lambda val:
                                                      'public'
                                                      if val== 'Public'
                                                      else 'others')

# acquiror_code & target_code (relatedness)

In [11]:
# Get relatedness from TRBC code
def relatedness(acquiror, target):
    if acquiror == target:
        return 'industry'
    elif acquiror[:6] == target[:6]:
        return 'industry_group'
    elif acquiror[:4] == target[:4]:
        return 'business_sector'
    elif acquiror[:2] == target[:2]:
        return 'economic_sector'
    else:
        return 'not_related'

In [12]:
clean['relatedness'] = clean.apply(lambda row:
                                   relatedness(str(row.acquiror_code), str(row.target_code)),
                                   axis=1)

# acquiror_code & target_code (industries)

## Declassifier

In [13]:
# transform code into different cols mentioned above
def declassifier(x):
    x = str(x)
    return x[:2], x[:4]

In [14]:
clean['economic_sector_ac'], clean['business_sector_ac'] = \
zip(*clean['acquiror_code'].map(declassifier))
clean['economic_sector_target'], clean['business_sector_target'] = \
zip(*clean['target_code'].map(declassifier))

In [15]:
clean.drop(columns=['acquiror_code', 'target_code'], inplace=True)

## Implementation of the declassifier function

In [16]:
# Implement the declassifier

pickle_in = open("/Users/Conradi/code/cobergmann/MA_PREDICTOR/MA_PREDICTOR/data/declassification.pkl","rb")
results = pickle.load(pickle_in)

In [17]:
def get_info_trbc(hierarchical_id):
    str_id= str(hierarchical_id)
    try:
        if len(str_id)==2:
            s = results.loc[results['hierarchical_id'].str.startswith(str_id), 'economic_sector'].reset_index(drop=True)
            return s[0]
        elif len(str_id)==4:
            r = results.loc[results['hierarchical_id'].str.startswith(str_id), 'business_sector'].reset_index(drop=True)
            return r[0]
        elif len(str_id)==6:
            q = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry_group'].reset_index(drop=True)
            return q[0] 
        elif len(str_id)==8:
            p = results.loc[results['hierarchical_id'].str.startswith(str_id), 'industry'].reset_index(drop=True)
            return p[0] 
        else:
            print("Please enter a valid hierarchical ID.")
    except:
        print("Please enter a valid hierarchical ID.")

In [18]:
clean['economic_sector_ac'] = clean.apply(lambda row: get_info_trbc(row.economic_sector_ac), axis=1)
clean['business_sector_ac'] = clean.apply(lambda row: get_info_trbc(row.business_sector_ac), axis=1)
clean['economic_sector_target'] = clean.apply(lambda row: get_info_trbc(row.economic_sector_target), axis=1)
clean['business_sector_target'] = clean.apply(lambda row: get_info_trbc(row.business_sector_target), axis=1)

# shares_acquired

In [19]:
clean['shares_acquired'] = clean['shares_acquired'].astype(str)

In [20]:
# Apply
clean['shares_acquired'] = clean['shares_acquired'].apply(lambda val:
                                                          "full"
                                                          if val=="100.0"
                                                          else "not_full")

# shares_at_announcement

In [21]:
# Transform
clean['shares_at_announcement'] = clean['shares_at_announcement'].astype(str)

# Apply
clean['shares_at_announcement'] = clean['shares_at_announcement'].apply(lambda val: "no" if val=="0.0" else "yes")

# consideration_offered

In [22]:
# transform pipes into list of all considerations
list_gen = lambda x: x.split('|')
clean['consideration_offered'] = clean.consideration_offered.apply(list_gen)

# Cluster considerations
def cash(considerations):
    
    for consideration in considerations:
        if 'Cash' not in consideration:
            return 'Other'
    return 'Cash'

# Apply
clean['consideration_offered'] = clean.apply(lambda row:
                                             cash(row.consideration_offered),
                                             axis=1)

# acquisition_technique (tbc with defined cluster of techniques)

In [23]:
# Transform string into list of techniques used
list_gen = lambda x: x.split('|')
clean['acquisition_technique'] = clean.acquisition_technique.apply(list_gen)

In [24]:
# Remove deals with fishy techniques (defined in cobergmann_detailed_data_exploration_advanced)

# Function to exclude fishy techniques
def fishy(techniques):
    
    # Define fishy techniques
    to_exclude = [
        'Open Market Purchase', # 3 deals will be lost
        'Sale and leaseback' # 16 deals will be lost
    ]
    
    # Label fishy techniques
    for technique in techniques:
        if technique in to_exclude:
            return 'exlclude'
    return 'ok'

# Apply detector
clean['red_flag'] = clean.apply(lambda row: fishy(row.acquisition_technique), axis=1)

# Remove fishy techniques
clean = clean[clean.red_flag == 'ok']

# Remove helper columns 'red_flag'
clean.drop(columns='red_flag', inplace=True)

# purpose (tbc with defined cluster of motives)

# acquiror_financial_advisor & target_financial_advisor (count)

**Top 10 M&A advisors sourced from dealroom (https://dealroom.net/blog/biggest-m-a-firms)**

In [25]:
# Defining top advisors

parties = ['acquiror_financial_advisor', 'target_financial_advisor']
adv = clean[parties].copy()

# Filling up NAs with 'Unknown'
adv.fillna('Unknown', inplace=True)

# unpack advisors and put them into set
advisors = set()
for index, row in adv.iterrows():
    for advisor in row['acquiror_financial_advisor'].split('|'):
        advisors.add(advisor)
    for advisor in row['acquiror_financial_advisor'].split('|'):
        advisors.add(advisor)
        
# Defining list of top M&A advisors ()
base = ['Barclays', 'Bank of America', 'Citi', 'Credit Suisse','Goldman Sachs',
        'JP Morgan', 'Lazard', 'Morgan Stanley', 'Rothschild', 'UBS']

# Adjust advisor's name to dataset (preliminary operation)
top_advisors = []
for advisor in advisors: # iterating through set of advisors
    for top_advisor in base:
        if top_advisor in advisor: # if (base) name of top_advisors is in set append
            top_advisors.append(advisor)
            
top_advisors = sorted(top_advisors)

# Manually investigate top_advisors and append/ remove accordingly
top_advisors.remove('Mitsubishi UFJ Morgan Stanley')

In [26]:
# Find out how many top_advisors were involved in transaction

# Filling up NAs with 'Unknown'
clean.fillna('Unknown', inplace=True)

# Transforming advisor pipe(s) into lists to iterate through
list_gen = lambda x: x.split('|')

for party in parties:
    clean[party] = clean[party].apply(list_gen)

# Counting number of top_advisors per deal (acquiror_side)
def counter(advisors):
    count = 0
    for advisor in advisors:
        if advisor in top_advisors:
            count += 1
    return count

# Create new cols for acquiror and target
clean['a_fin_adv_count'] = clean.apply(lambda row: counter(row.acquiror_financial_advisor), axis=1)
clean['t_fin_adv_count'] = clean.apply(lambda row: counter(row.target_financial_advisor), axis=1)

# Drop cols
clean.drop(columns=parties, inplace=True)

# Clean-up

In [27]:
clean.drop(columns=['acquisition_technique', 'purpose'], inplace=True) # Comment if necessary

In [28]:
old_order = clean.columns.tolist()
new_order = old_order[:5] + old_order[6:] + [old_order[5]]

In [29]:
clean = clean[new_order]

In [30]:
clean.shape

(3293, 16)

In [31]:
clean.head()

,announcement_date,target_status,shares_at_announcement,shares_acquired,consideration_offered,month,rel_deal_value,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,a_fin_adv_count,t_fin_adv_count,car
0,2005-01-01,others,no,full,Cash,1,0.003797,cross_border,business_sector,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,0,0,0.006854
1,2005-01-04,public,no,full,Cash,1,0.024922,cross_border,not_related,Consumer Non-Cyclicals,Food & Drug Retailing,Industrials,Transportation,1,0,-0.009362
2,2005-01-11,public,no,full,Other,1,0.008012,cross_border,industry,Financials,Banking & Investment Services,Financials,Banking & Investment Services,0,0,-0.011249
3,2005-01-12,public,no,not_full,Cash,1,0.382792,national,industry,Basic Materials,Mineral Resources,Basic Materials,Mineral Resources,0,0,0.013061
4,2005-01-14,others,no,full,Cash,1,0.009290,cross_border,not_related,Industrials,Industrial Goods,Basic Materials,Mineral Resources,0,0,0.000000


# Uploading

In [32]:
clean.to_csv('../MA_PREDICTOR/data/ma_detailed_data_car_clean.csv', index=False, date_format="%d/%m/%Y")